In [45]:
import os
import joblib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector, TimeDistributed, Dense, LayerNormalization, SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping

In [46]:
MODEL_DIR = "models"
os.makedirs(MODEL_DIR, exist_ok=True)

In [47]:
import numpy as np
import pandas as pd
import os
import joblib
import gc

# =========================================
# CẤU HÌNH
# =========================================
FOLDER_NAME = "processed_data_final_v1"
CSV_FILENAME = "train_data_optimized.csv"
FEATURE_NAMES_FILE = "final_feature_list.joblib"

SEQ_LEN = 10
GROUP_ID_COL = 'card1'
TIME_COL = 'TransactionDT'
TARGET_COL = 'isFraud'

# =========================================
# HÀM HỖ TRỢ
# =========================================
def create_sequences(data, seq_len):
    """Tạo mảng 2D thành các chuỗi 3D."""
    sequences = []
    if len(data) >= seq_len:
        for i in range(len(data) - seq_len + 1):
            window = data[i:(i + seq_len)]
            sequences.append(window)
    return sequences

# =========================================
# QUY TRÌNH CHÍNH
# =========================================
print("🚀 BẮT ĐẦU TẠO CHUỖI 3D (CHIẾN LƯỢC: CLEAN USERS ONLY)")

# 1. Tải dữ liệu
csv_path = os.path.join(FOLDER_NAME, CSV_FILENAME)
features_path = os.path.join(FOLDER_NAME, FEATURE_NAMES_FILE)
if not os.path.exists(csv_path): raise FileNotFoundError("❌ Thiếu file CSV.")

print("👉 Đang đọc dữ liệu...")
df_all = pd.read_csv(csv_path)
final_feature_names = joblib.load(features_path)

# --- BƯỚC 2 ĐƯỢC NÂNG CẤP ---
print("👉 Đang xác định các thẻ 'sạch' (chưa từng bị gian lận)...")
# Tìm danh sách các thẻ đã từng có ít nhất 1 giao dịch gian lận
fraud_cards = df_all[df_all[TARGET_COL] == 1][GROUP_ID_COL].unique()
print(f"   - Đã tìm thấy {len(fraud_cards):,} thẻ có dính líu đến gian lận.")

# Chỉ giữ lại các thẻ KHÔNG nằm trong danh sách đen này
df_clean_users = df_all[~df_all[GROUP_ID_COL].isin(fraud_cards)].copy()
print(f"   - Số lượng giao dịch ban đầu: {len(df_all):,}")
print(f"   - Số lượng giao dịch sau khi lọc sạch: {len(df_clean_users):,}")
print(f"   - Tỷ lệ dữ liệu giữ lại: {len(df_clean_users)/len(df_all)*100:.2f}%")

del df_all; gc.collect() # Dọn dẹp dữ liệu gốc

# 3. Chuẩn bị columns
feature_cols = [c for c in final_feature_names if c not in [GROUP_ID_COL, TIME_COL, TARGET_COL]]
cols_to_use = [GROUP_ID_COL, TIME_COL] + feature_cols
# Sử dụng df_clean_users thay vì df_normal cũ
df_processing = df_clean_users[cols_to_use] 

# 4. Tạo chuỗi (Giữ tất cả trên RAM)
print(f"👉 Đang xử lý nhóm theo '{GROUP_ID_COL}' để tạo chuỗi...")
grouped = df_processing.groupby(GROUP_ID_COL)

all_sequences = []
count = 0
for group_id, group_df in grouped:
    group_df_sorted = group_df.sort_values(by=TIME_COL)
    feature_data = group_df_sorted[feature_cols].values.astype(np.float32)
    
    seqs = create_sequences(feature_data, SEQ_LEN)
    all_sequences.extend(seqs)
    
    count += 1
    if count % 1000 == 0: print(f"   - Đã xử lý {count} nhóm...", end='\r')

print(f"\n👉 Đã xử lý xong {count} nhóm. Đang chuyển đổi sang numpy array...")
del df_processing, df_clean_users, grouped; gc.collect()

# 5. Chuyển list thành numpy array 3D
if all_sequences:
    X_train_3d = np.array(all_sequences, dtype=np.float32)
    print("\n✅ HOÀN TẤT!")
    print(f"🎯 X_train_3d shape: {X_train_3d.shape}")
    # In thêm thông tin để kiểm tra
    print(f"   - Số mẫu huấn luyện: {X_train_3d.shape[0]:,}")
    print(f"   - Độ dài chuỗi (Timesteps): {X_train_3d.shape[1]}")
    print(f"   - Số đặc trưng (Features): {X_train_3d.shape[2]}")
    del all_sequences; gc.collect()
else:
    print("\n❌ CẢNH BÁO: Không tạo được chuỗi nào (có thể SEQ_LEN quá lớn).")
    X_train_3d = np.empty((0, SEQ_LEN, len(feature_cols)), dtype=np.float32)

🚀 BẮT ĐẦU TẠO CHUỖI 3D (CHIẾN LƯỢC: CLEAN USERS ONLY)
👉 Đang đọc dữ liệu...
👉 Đang xác định các thẻ 'sạch' (chưa từng bị gian lận)...
   - Đã tìm thấy 1,532 thẻ có dính líu đến gian lận.
   - Số lượng giao dịch ban đầu: 472,432
   - Số lượng giao dịch sau khi lọc sạch: 117,094
   - Tỷ lệ dữ liệu giữ lại: 24.79%
👉 Đang xử lý nhóm theo 'card1' để tạo chuỗi...
   - Đã xử lý 11000 nhóm...
👉 Đã xử lý xong 11198 nhóm. Đang chuyển đổi sang numpy array...

✅ HOÀN TẤT!
🎯 X_train_3d shape: (67649, 10, 98)
   - Số mẫu huấn luyện: 67,649
   - Độ dài chuỗi (Timesteps): 10
   - Số đặc trưng (Features): 98


In [48]:
from sklearn.model_selection import train_test_split

X_train, X_val = train_test_split(X_train_3d, test_size=0.2, random_state=42)

print(f"Tổng số chuỗi: {len(X_train_3d)}")
print(f"Số chuỗi huấn luyện (X_train): {len(X_train)}")
print(f"Số chuỗi kiểm tra (X_val): {len(X_val)}")
print("---")
print(f"Hình dạng X_train: {X_train.shape}")
print(f"Hình dạng X_val: {X_val.shape}")

Tổng số chuỗi: 67649
Số chuỗi huấn luyện (X_train): 54119
Số chuỗi kiểm tra (X_val): 13530
---
Hình dạng X_train: (54119, 10, 98)
Hình dạng X_val: (13530, 10, 98)


In [49]:
print("Min value in X_train:", np.min(X_train))
print("Max value in X_train:", np.max(X_train))

Min value in X_train: 0.0
Max value in X_train: 1.0


In [50]:
print(X_train_3d.shape[1])
print(X_train_3d.shape[2])

10
98


In [51]:
SEQ_LEN = X_train_3d.shape[1]
INPUT_DIM = X_train_3d.shape[2]
print(SEQ_LEN, INPUT_DIM)

10 98


In [52]:
import sys
import os
sys.path.append(os.path.abspath(".."))
from train_model_V3 import LSTMAutoencoderTrainer

In [53]:
# Khởi tạo trainer
trainer = LSTMAutoencoderTrainer(seq_len=SEQ_LEN, input_dim=INPUT_DIM)

trainer.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 10, 98)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_lstm_1 (LSTM)           │ (None, 10, 64)         │        41,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_4             │ (None, 10, 64)         │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_12          │ (None, 10, 64)         │           128 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_lstm_2 (LSTM)           │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_13          │ (None, 64)             │           128 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ latent_vector (RepeatVector)    │ (None, 10, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_lstm (LSTM)             │ (None, 10, 64)         │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_14          │ (None, 10, 64)         │           128 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (TimeDistributed)  │ (None, 10, 98)         │         6,370 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 114,530 (447.38 KB)

 Trainable params: 114,530 (447.38 KB)

 Non-trainable params: 0 (0.00 B)

In [54]:
history = trainer.fit(
    X_train=X_train,
    X_val=X_val,
    epochs=100,
    batch_size=128,
    patience=10           # số epoch chờ trước khi dừng sớm
)

Epoch 1/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - loss: 0.1376 - val_loss: 0.0467
Epoch 2/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - loss: 0.0390 - val_loss: 0.0322
Epoch 3/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - loss: 0.0317 - val_loss: 0.0282
Epoch 4/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - loss: 0.0300 - val_loss: 0.0276
Epoch 5/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - loss: 0.0296 - val_loss: 0.0270
Epoch 6/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - loss: 0.0292 - val_loss: 0.0267
Epoch 7/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - loss: 0.0286 - val_loss: 0.0263
Epoch 8/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - loss: 0.0281 - val_loss: 0.0251
Epoch 9/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - loss: 0.0275 - val_loss: 0.0249
Epoch 10/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - loss: 0.0273 - val_loss: 0.0247
Epoch 11/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - loss: 0.0268 - val_loss: 0.0241
Epoch 12/100
423/423 ━━━━━━━━

In [55]:
trainer.save_model("models/lstm_isFraud.keras")

✅ Model saved to models/lstm_isFraud.keras
